In [ ]:
import requests
from pygltflib import GLTF2
from io import BytesIO
import tempfile
import numpy as np
from PIL import Image
import base64
import pandas as pd

# Function to fetch the GLB file content
def fetch_glb_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        return BytesIO(response.content)
    else:
        raise Exception(f"Failed to fetch file: {response.status_code}")

# Function to load and parse the GLB file from content
def load_glb_from_content(content):
    with tempfile.NamedTemporaryFile(delete=True, suffix=".glb") as temp_file:
        temp_file.write(content.read())
        temp_file.flush()
        gltf = GLTF2().load_binary(temp_file.name)
    return gltf

# Function to extract the 6th texture image from the GLB file (index 5)
def extract_sixth_texture_image(gltf):
    if len(gltf.textures) > 5:
        image_index = gltf.textures[5].source  # 6th texture (index 5)
        image = gltf.images[image_index]
        
        if image.uri:
            # Decode the base64 data URI
            image_data = decode_base64_data_uri(image.uri)
        else:
            # Handle embedded binary data
            buffer_view = gltf.bufferViews[image.bufferView]
            buffer = gltf.buffers[buffer_view.buffer]
            data = gltf.get_data_from_buffer_uri(buffer.uri)
            start = buffer_view.byteOffset
            end = start + buffer_view.byteLength
            image_data = data[start:end]
        
        # Save and return the image
        with open('sixth_texture.png', 'wb') as f:
            f.write(image_data)
        return 'sixth_texture.png'
    else:
        raise Exception("Less than 6 textures in the GLB file")

# Function to decode base64 data URIs
def decode_base64_data_uri(data_uri):
    header, encoded = data_uri.split(",", 1)
    data = base64.b64decode(encoded)
    return data

# Function to fetch an image from a file path
def fetch_image_from_file(image_path):
    try:
        return Image.open(image_path)
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

# Function to compare two images based on color histograms
def compare_images(image1, image2):
    image1 = image1.resize((100, 100)).convert('RGB')  # Resize and convert to RGB
    image2 = image2.resize((100, 100)).convert('RGB')
    
    hist1 = np.array(image1.histogram())
    hist2 = np.array(image2.histogram())
    
    # Compare histograms using Chi-Square method
    chi_square = 0.5 * np.sum(((hist1 - hist2) ** 2) / (hist1 + hist2 + 1e-10))
    return chi_square

# Example usage
glb_url = "https://models.readyplayer.me/66a54feb8409082e90051729.glb"

# Fetch the GLB content
glb_content = fetch_glb_content(glb_url)

# Load and parse the GLB file from content
gltf = load_glb_from_content(glb_content)

# Extract the 6th texture image
sixth_texture_image_path = extract_sixth_texture_image(gltf)
extracted_texture_image = fetch_image_from_file(sixth_texture_image_path)

# Load dataset
dataset_csv = 'myntra.csv'  # Update with the correct path to your CSV file
df = pd.read_csv(dataset_csv)

best_score = float('inf')
best_match = None

# Compare each image in the dataset with the extracted texture image
for index, row in df.iterrows():
    try:
        image_url = row['img']  # Update with the correct column name in your CSV
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content))
        
        if image is not None:
            similarity_score = compare_images(extracted_texture_image, image)
            if similarity_score < best_score:
                best_score = similarity_score
                best_match = image_url
    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Output the best match
print("Best match found:")
print(f"Image URL: {best_match}")
print(f"Similarity Score: {best_score}")

Best match found:
Image: /home/vanshika/Desktop/images/image.jpeg
Similarity Score: 83.794
